# Un semplice filtro RC passa-basso

<img src="figs/filtro_rc/RC_filter.png" width="400"/>

Il circuito in alto è descritto dall'equazione di stato

$$
\frac{\mathrm{d}v_{out}}{\mathrm{dt}} = - \frac{v_{out}}{RC} + \frac{v_{in}(t)}{RC} ,
$$

la cui soluzione può essere scritta come

$$
v_{out}(t) = K e^{-\frac{t}{RC}} + v_{out}^{ip}(t) ,
$$

dove K è una costante che dipende dalla condizione iniziale e $v_{out}^{ip}(t)$ è l'integrale particolare, che in generale sarà funzione dell'ingresso $v_{in}(t)$.

Consideriamo quindi un ingresso sinusoidale del tipo $v_{in}(t) = E \cos(\omega t + \phi)$: passando al dominio dei fasori abbiamo visto che possiamo scrivere $v_{out}^{ip}(t)$ come

$$
v_{out}^{ip}(t) = \frac{E}{\sqrt{1+\omega^2 R^2 C^2}}\ \cos\left(\omega t + \phi + \theta\right) ,
$$

dove $\theta = \arctan(-\omega R C)$ è lo sfasamento introdotto dalla presenza del condensatore.

La soluzione complessiva è quindi data da:

$$
v_{out}(t) = \underbrace{\left(v_{out}(0) - \frac{E \cos(\phi + \theta)}{\sqrt{1+\omega^2 R^2 C^2}}\right)\  e^{-\frac{t}{RC}}}_{v_{out}^{rl}(t)} + \underbrace{\frac{E}{\sqrt{1+\omega^2 R^2 C^2}}\ \cos\left(\omega t + \phi + \theta\right)}_{v_{out}^{ip}(t)} .
$$

Il termine di risposta libera si smorza con una costante di tempo $\tau = RC$, mentre la risposta forzata sarà tanto più attenuata quanto è elevata la pulsazione $\omega$ del segnale di ingresso.

In questo notebook vedremo come analizzare numericamente il filtro passa-basso mostrato nella figura in alto.

In [ ]:
from __future__ import print_function

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', size=12)
import seaborn as sns

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline

Definisco innanzitutto le costanti del problema

In [ ]:
R = 1e3        # [Ohm]
C = 1e-6       # [F]
E = 1          # [V]
phi = 0        # [1]
# la condizione iniziale
vout0 = [0]     # [V]
print('La costante di tempo del circuito è tau = {} ms.'.format(R*C*1e3))

In [ ]:
def compute_analytical_solution(w, E, R, C, phi, v0, t_span):
    t = np.linspace(t_span[0], t_span[1], 10000)
    # lo sfasamento tra le due sinusoidi
    theta = np.arctan(-w*R*C)
    # la costante K della soluzione
    K = v0 - E / np.sqrt(1 + (w*R*C)**2) * np.cos(phi + theta)
    # l'ingresso v_in(t)
    vin = E * np.cos(w*t + phi)
    # l'espressione di v_out(t)
    vout = K * np.exp(-t/(R*C)) + E / np.sqrt(1 + (w*R*C)**2) * np.cos(w*t + phi + theta)
    return t,vin,vout

#### Definisco una funzione che calcola v_out data la frequenza $f$ della sinusoide in ingresso

In [ ]:
@interact(log_f=(np.log(1), np.log(2000)))
def plot_analytical_solution(log_f):
    ### Definizioni iniziali
    f = np.exp(log_f)
    # pulsazione di oscillazione
    w = 2*np.pi*f  # [rad/s]
    # estremi di integrazione: calcolo la soluzione su 10 periodi della sinusoide di ingresso
    t_span = np.array([0,10/f])
    
    ### Calcolo la soluzione esatta dell'equazione di stato
    t,vin,vout = compute_analytical_solution(w, E, R, C, phi, vout0[0], t_span)
    
    ### Visualizzo i risultati
    fig,ax = plt.subplots(1, 1, figsize=(6,4))
    if t_span[1] > 0.5:
        coeff = 1
        unit = 's'
    else:
        coeff = 1e3
        unit = 'ms'
    ax.plot(t*coeff, vin, color=[.6,.6,.6], lw=3, label=r'$v_{in}(t)$')
    ax.plot(t*coeff, vout, color=[1, 0, .5], lw=2, label=r'$v_{out}(t)$')
    ax.plot(t_span*coeff, E/np.sqrt(1+(w*R*C)**2)+np.zeros(2), '--', \
            color=[0,.8,0], lw=1, label=r'$\frac{E}{\sqrt{1+(\omega R C)^2}}$')
    ax.plot(t_span*coeff, -E/np.sqrt(1+(w*R*C)**2)+np.zeros(2), '--', \
            color=[0,.8,0], lw=1)
    ax.legend(loc='upper right', fontsize=18)
    ax.set_xlabel(f'Time ({unit})', fontsize=14)
    ax.set_ylabel('Voltage (V)', fontsize=14)
    ax.set_title(f'F = {f:g} Hz')
    yticks = np.linspace(-1,1,5)
    ax.set_yticks(yticks)
    sns.despine()
    fig.tight_layout()

Fissiamo ora $f=100\ Hz$ e integriamo il sistema usando una funzione per la soluzione numerica di sistemi di equazioni differenziali del primo ordine

In [ ]:
# definisce la rhs dell'equazione di stato
def odefun(t, vout, R, C, E, f, phi):
    return (-vout + E * np.cos(2*np.pi*f*t + phi)) / (R*C)

# frequenza di oscillazione
f = 100 # [Hz]
w = 2 * np.pi * f
# condizione iniziale
vout0 = [0]
# estremi di integrazione
t_span = [0,10/f]

# tolleranze per l'integrazione
atol = 1e-10
rtol = 1e-8

t,vin,vout = compute_analytical_solution(w, E, R, C, phi, vout0[0], t_span)

# integro numericamente l'equazione di stato
sol = solve_ivp(lambda t,x: odefun(t,x,R,C,E,f,phi), t_span, vout0, \
                method='BDF', atol=atol, rtol=rtol)

Verifichiamo che la soluzione numerica sia uguale a quella analitica

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6,4))
ax.plot(sol['t'], sol['y'][0], color=[.6,.6,.6], lw=3, label=r'$v_{out}(t)$ numerica')
ax.plot(t, vout, '--', color=[1, 0, .5], lw=2, label=r'$v_{out}(t)$ analitica')
ax.legend(loc='lower right', fontsize=14)
ax.set_xlabel('Time (s)', fontsize=14)
ax.set_ylabel('Voltage (V)', fontsize=14)
yticks = np.linspace(-1,1,5)
ax.set_yticks(yticks)
sns.despine()
fig.tight_layout()

Vediamo ora come variano l'attenuazione e lo sfasamento di $v_{out}(t)$ al variare della frequenza $f$ di $v_{in}(t)$

In [ ]:
# frequenze da 10^0 a 10^5 equispaziate logaritmicamente
f = np.logspace(0, 5, 100)
# le corrispondenti pulsazione
w = 2*np.pi*f
# l'ampiezza della risposta forzata
amplitude = E / np.sqrt(1 + (w*R*C)**2)
# lo sfasamento
theta = np.arctan(-w*R*C)
# visualizzo i risultati
fig,(ax1,ax2) = plt.subplots(1, 2, figsize=(8,3))
ax1.semilogx(f, amplitude, 'k', lw=2)
ax1.plot(E/(R*C)+np.zeros(2), [0,1], 'r--', lw=2, label=r'1/$\tau$')
ax1.legend(loc='lower left')
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel(r'Max ( $v_{out}(t)$ ) ')
ax2.semilogx(f, np.rad2deg(theta), 'k', lw=2)
ax2.plot(E/(R*C)+np.zeros(2), [-90,0], 'r--', lw=2, label=r'1/$\tau$')
ax2.legend(loc='lower left')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel(r'$\theta$')
sns.despine()
fig.tight_layout()